In [ ]:
!pip install pydub

In [ ]:
df = pd.read_csv('birdsongonlinefinal2021.csv')
df.head(1000)

In [ ]:
%pip install pydub
import pandas as pd
import requests
from pydub import AudioSegment
import os
from tqdm import tqdm

# Create a directory to store the audio files 
os.makedirs('audio_files', exist_ok=True)

# Modify url to include extension
df['URL'] = df['URL'] + '.mp3'

# Function to download and save audio from URL
def download_and_save_audio(url, filename):
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)

# Iterate over the URLs and download the audio files
for i, url in tqdm(enumerate(df['URL']), total=len(df['URL'])):
    filename = f'audio_files/audio_{i}.mp3'
    download_and_save_audio(url, filename)

# Store the paths to the audio files in the DataFrame
df['audio_path'] = [f'audio_files/audio_{i}.mp3' for i in range(len(df['URL']))]

df.head(253)

Note: you may need to restart the kernel to use updated packages.


  0%|          | 0/253 [00:00<?, ?it/s]

  0%|          | 1/253 [00:06<26:01,  6.20s/it]

  1%|          | 2/253 [00:12<27:07,  6.49s/it]

  1%|          | 3/253 [00:19<26:47,  6.43s/it]

...

 13%|█▎        | 34/253 [03:16<22:23,  6.13s/it]

 14%|█▍        | 35/253 [03:23<22:47,  6.27s/it]

 14%|█▍        | 36/253 [03:28<21:35,  5.97s/it]

 15%|█▍        | 37/253 [03:35<22:55,  6.37s/it]

 15%|█▌        | 38/253 [03:41<22:09,  6.18s/it]

In [ ]:
import re

# Remove the common prefix
df['Summary'] = df['Summary'].str.replace('John Hutchinson is an ornithologist and a birdcall recordist.', '')

# Function to extract location from the summary
def extract_location(summary):
    location = re.search('made at (.+?)\.', summary)
    if location:
        return location.group(1)
    location = re.search('made on (.+?)\.', summary)
    if location:
        return location.group(1)
    location = re.search('at (.+?)\.', summary)
    if location:
        return location.group(1)
    location = re.search('in (.+?)\.', summary)
    if location:
        return location.group(1)
    return ''

# Apply the function to the 'Summary' column
df['location'] = df['Summary'].apply(extract_location)

df.head(1000)

In [ ]:
import re
import datetime

# Function to extract date from the summary
def extract_date(summary):
    date = re.search('\d{1,2} \w+ \d{4}', summary)
    if date:
        return date.group(0)
    date = re.search('\w+ \d{4}', summary)
    if date:
        return datetime(f' 01 {date.group(0)}')
    return None

# Apply the function to the 'Summary' column
df['date'] = df['Summary'].apply(extract_date)

# Convert the 'Subject headings: Topical' into a list
df['Subject headings: Topical'] = df['Subject headings: Topical'].str.split(';')

df.head(253)

In [ ]:
import requests

def get_gbif_species(common_name):
    url = f'https://api.gbif.org/v1/species/match?name={common_name}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('scientificName')
    else:
        return None

# Example usage:
print(get_gbif_species('House Sparrow'))

In [ ]:
def get_lat_long(location):
    url = f'https://nominatim.openstreetmap.org/search?format=json&q={location}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]['lat'], data[0]['lon']
    return None, None

# Example usage:
print(get_lat_long('New York'))


In [ ]:
# Create a new column 'scientific_names' by applying the get_gbif_species function to each item in the 'Subject headings: Topical' lists
df['scientific_names'] = df['Subject headings: Topical'].apply(lambda x: [get_gbif_species(item) for item in x])

# Create new columns 'latitude' and 'longitude' by applying the get_lat_long function to the 'Location' column
df[['latitude', 'longitude']] = df['Location'].apply(get_lat_long).apply(pd.Series)
